<a href="https://colab.research.google.com/github/mini3713/mini3713/blob/main/occupancy/def_zip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### bar chart

In [ ]:
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='whitegrid')

plt.rcParams["figure.figsize"] = (10,7)
plt.rc('font', size=10)
plt.rc('axes', labelsize=15)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=11)
plt.rc('figure', titlesize=15)

In [ ]:
bar = sns.countplot(x="회차", data=new_df)
bar.set_xticklabels(['여름','가을','겨울'])

for p in bar.patches:
  bar.annotate("{:,}".format(int(p.get_height())), (p.get_x()+ p.get_width()/2, p.get_height()), ha='center', va='center', size=10, xytext=(0,6), textcoords='offset points')

### 천단위 콤마 표시(y축)
def yfunc(y, pos):
    s = '{:0,d}'.format(int(y))
    return s
y_format = tkr.FuncFormatter(yfunc)
bar.yaxis.set_major_formatter(y_format)

### line chart by column

In [ ]:
### column별 분류에 따른 평균을 꺾은선그래프로 시각화해주는 함수
def visualize_column_unique(k):    
 
  data = new_df
  oh = data.iloc[:,6:]
  ts = pd.date_range('2019-07-01 00:00:00', '2019-07-01 23:50:00', freq='10min')

  # 원하는 칼럼 선택/고유값 추출
  column = data.iloc[:,k]
  unique_list = data.iloc[:,k].unique()

  plt.figure(figsize=(18,4))
  plt.ylim([0, 1.0]) 

  for i in unique_list: 

    # 고유값에 따른 라벨 추출 
    col_val = column[(column == i)]
    label = col_val.index.tolist()    

    # 시계열 데이터에 라벨 전달하여 데이터 추출
    # 고유값에 따른 시계열 평균 계산  
    oh_label = oh.loc[label]
    oh_rate = oh_label.sum(axis=0) / len(oh_label)
    
    plt.plot(ts, oh_rate, label=f'{i}')
    plt.xlabel('Time')
    plt.ylabel('Occupancy')
    plt.legend()

In [ ]:
### cluster label에 따른 평균을 꺾은선그래프로 시각화해주는 함수
def visualize_km_dwt(k):    
  
  # dwt한 데이터로 KMeans
  km = KMeans(n_clusters = k, max_iter=2, random_state=42)
  cluster = km.fit(n_cD5)
  n_cD5['cluster'] = cluster.labels_

  ts = pd.date_range('2019-07-01 00:00:00', '2019-07-01 23:50:00', freq='10min')
  
  plt.figure(figsize=(18,4))

  for i in range (0, k, 1):
    
    # 클러스터 라벨에 따른 인덱스를 list로 저장
    cluster_label = n_cD5.index[(n_cD5['cluster'] == i)].tolist()

    # 원핫인코딩 파일로 인덱스전달 > 라벨별 평균 점유율 계산
    df_label = df.loc[cluster_label]
    df_rate = df_label.sum(axis=0) / len(df_label)
    
    plt.plot(ts, df_rate, label=f'{i}')
    plt.xlabel('Time')
    plt.ylabel('Occupancy')
    plt.legend()

### silhouette_score

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성
### k-means가 아닌 time-series k-means로 작성함 
def visualize_silhouette_KM(cluster_lists, X_features): 

    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 계수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 계수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=2, random_state=42)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([-0.4, -0.2, 0, 0.2, 0.4, 0.6])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")